In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tensorflow.keras.applications import ResNet50

In [3]:
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import load_model

In [4]:
import numpy as np
from sklearn import metrics

In [5]:
from experiments.data_gen import PascalVOCDataGenerator

Using TensorFlow backend.


In [6]:
from model.callbacks.metric_callbacks import MAPCallback
from model.callbacks.save_callback import SaveModel

In [7]:
from model.losses import BCE

In [8]:
prop=70

## model

In [ ]:
model = ResNet50(include_top=True, weights='imagenet')
model.layers.pop()
model = Model(input=model.input, output=model.layers[-1].output)

In [ ]:
model.summary()
model.compile(loss='binary_crossentropy', optimizer=SGD(lr, momentum=0.9), metrics=['binary_accuracy'])

## data

In [9]:
data_dir = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/'
data_generator_train = PascalVOCDataGenerator('trainval', data_dir, prop=prop)

loading dataset from /share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/Annotations/annotations_multilabel_trainval_partial_70_1.csv


In [ ]:
batch_size = 32
generator = data_generator_train.flow(batch_size=batch_size)

In [ ]:

# Initilisation des matrices contenant les Deep Features et les labels
X_train = np.zeros((len(data_generator_train.images_ids_in_subset),2048))
Y_train = np.zeros((len(data_generator_train.images_ids_in_subset),20))

# Calcul du nombre e batchs
nb_batches = int(len(data_generator_train.images_ids_in_subset) / batch_size) + 1

for i in range(nb_batches):
    # Pour chaque batch, on extrait les images d'entrée X et les labels y
    X, y = next(generator)
    # On récupère les Deep Feature par appel à predict
    y_pred = model.predict(X)
    X_train[i*batch_size:(i+1)*batch_size,:] = y_pred
    Y_train[i*batch_size:(i+1)*batch_size,:] = y

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(20,  input_dim=2048, name='fc1', activation='sigmoid'))
model.summary()

In [ ]:
learning_rate = 0.1
sgd = SGD(learning_rate)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['binary_accuracy'])

In [ ]:
model.fit(X_train, Y_train,batch_size=batch_size, epochs=nb_epoch,verbose=1)
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s TEST: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("%s TEST: %.2f%%" % (model.metrics_names[1], scores[1]*100))

## Eval

In [10]:
data_generator_test = PascalVOCDataGenerator('test', data_dir)
len(data_generator_test.images_ids_in_subset)

4952

In [11]:
batch_size = len(data_generator_test.images_ids_in_subset)
generator_test = data_generator_test.flow(batch_size=batch_size)

In [12]:
X_test, Y_test = next(generator_test)

In [ ]:
print(X_test.shape, Y_test.shape)

In [ ]:
def eval_fn(model, X_test, Y_test):
    y_pred_test = model.predict(X_test)
    #y_pred_train = model.predict(X_train)

    #AP_train = np.zeros(20)
    AP_test = np.zeros(20)
    for c in range(20):
        #AP_train[c] = average_precision_score(Y_train[:, c], y_pred_train[:, c])
        AP_test[c] = metrics.average_precision_score(Y_test[:, c], y_pred_test[:, c])

    #print "MAP TRAIN =", AP_train.mean()*100
    print("MAP TEST =", AP_test.mean()*100)
    print(AP_test)

In [ ]:
eval_fn(model, X_test, Y_test)

In [ ]:
print(AP_test)

## Finetune

In [ ]:
# vanilla from RCP209
model = ResNet50(include_top=True, weights='imagenet')
model.layers.pop()
# Modify top layers
x = model.layers[-1].output
x = Dense(data_generator_train.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.input, outputs=x)

In [13]:
# Laura way
# Load ResNet50 architecture & its weights
input_shape = (224, 224, 3)
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

inp = Input(shape=input_shape, name='image_input')
x = resnet(inp)
x = Flatten()(x)
output = Dense(data_generator_train.nb_classes, activation='sigmoid')(x)
model = Model(inputs=inp, outputs=output)

/home/caleml/.local/lib/python3.4/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 20)                2007060   
Total params: 25,594,772
Trainable params: 25,541,652
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
lr = 0.1
# model.compile(loss='binary_crossentropy', optimizer=SGD(lr=lr), metrics=['binary_accuracy'])
loss = BCE()
model.compile(loss=loss, optimizer=SGD(lr=lr), metrics=['binary_accuracy'])

In [ ]:
# callbacks
exp_folder = '/home/caleml/partial_experiments/exp_20190628_0258_TESTNB'

cb_list = list()
cb_list.append(SaveModel(exp_folder, prop))

map_cb = MAPCallback(X_test, Y_test, exp_folder)
cb_list.append(map_cb)

In [ ]:
batch_size=32
nb_epochs=20
data_generator_train = PascalVOCDataGenerator('trainval', data_dir, prop=prop)
steps_per_epoch_train = int(len(data_generator_train.id_to_label) / batch_size) + 1
model.fit_generator(data_generator_train.flow(batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch_train,
                    epochs=nb_epochs,
                    callbacks=cb_list,
                    verbose=1)

loading dataset from /share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/Annotations/annotations_multilabel_trainval_partial_70_1.csv
Epoch 1/20
156/157 [============================>.] - ETA: 0s - loss: 0.6438 - binary_accuracy: 0.3092
Trying to save model @epoch=001 to /home/caleml/partial_experiments/exp_20190628_0258_TESTNB/model_70_001.h5
Couldn't save model, saving weights instead at /home/caleml/partial_experiments/exp_20190628_0258_TESTNB/weights_70_001.h5
type true <class 'numpy.ndarray'>, type pred <class 'numpy.ndarray'>
ap scores type <class 'numpy.ndarray'>
interval evaluation - epoch: 0 - mAP score: 0.136883
157/157 [==============================] - 87s 553ms/step - loss: 0.6415 - binary_accuracy: 0.3093
Epoch 2/20
156/157 [============================>.] - ETA: 0s - loss: 0.2289 - binary_accuracy: 0.3134
Trying to save model @epoch=002 to /home/caleml/partial_experiments/exp_20190628_0258_TESTNB/model_70_002.h5
Couldn't save model, saving weights instead at /home/c

## Dataset comparison

In [ ]:
data_dir = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/'
data_generator_train = PascalVOCDataGenerator('trainval', data_dir)

In [ ]:
nico_data = data_generator_train.id_to_label
print(len(nico_data))
print(nico_data['000131'])

In [ ]:
trainval_path = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/Annotations/annotations_multilabel_trainval.csv'

In [ ]:
laura_data = dict()
with open(trainval_path, 'r') as f_in:
    for line in f_in:
        parts = line.strip().split(',')
        laura_data[parts[0]] = [int(elt) for elt in parts[1:]]

In [ ]:
print(len(laura_data))
print(laura_data['000131'])

In [ ]:
for id_img, labels in nico_data.items():
    laura_labels = laura_data[id_img]
    converted = [l if l in [0, 1] else 0 for l in laura_labels]
    assert all([converted[i] == labels[i] for i in range(len(labels))])